# 1-)K-Nearest Neigbors  Regression Model (KNN)

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')


import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from scipy.stats import kurtosis, skew
plt.style.use("seaborn-darkgrid")


def K_Nearest_Neighbors_Regression_Model(independent_variables, target_variable):
    print("*************************************************K Nearest Neighbors Regression Model*********************************************************")
    
    def optimum_number_of_neighbors(independent_variables, target_variable):
        X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
        knn_params = {'n_neighbors': np.arange(1,30,1)}
        knn = KNeighborsRegressor()
        knn_cv_model = GridSearchCV(knn, knn_params, cv = 10)
        knn_cv_model.fit(X_train, y_train)
        optimum_number_of_neighbors=knn_cv_model.best_params_["n_neighbors"]
        print("Optimum number of neighbors is {}".format(optimum_number_of_neighbors))
        
        return optimum_number_of_neighbors
              
    a=optimum_number_of_neighbors(independent_variables, target_variable) 
    
    X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
    knn_model=KNeighborsRegressor(n_neighbors = a).fit(X_train, y_train)
    
    def optimum_root_mean_square_error( knn_model, X_train, X_test, y_train, y_test):# Bu kismda  diger  modelerle karsilastirip en iyi modeli secmek icin  opitimum rmse kullanacagiz  
        optimum_rmse_test=np.sqrt(-cross_val_score( knn_model, X_test, y_test, cv = 10, scoring = "neg_mean_squared_error")).mean()#K KATLI cross validtion  uygulama yaptiktan sonnraki TEST hatamiz
        optimum_rmse_train=np.sqrt(-cross_val_score( knn_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")).mean()#K KATLI cross validtion  uygulama yaptiktan sonnraki TRAIN hatamiz
        df1=pd.DataFrame(optimum_rmse_test, index=["optimum_RMSE_test"],columns=['Coefficient'])
        df2=pd.DataFrame(optimum_rmse_train, index=["optimum_RMSE_train"],columns=['Coefficient'])
        df_concat=pd.concat([df2,df1],axis=0)
        return df_concat
    
    
    def optimum_R_squred_Score(knn_model ,X_train, y_train):#R^2 (R-square degeridir).#Yani bagimsiz degiskenler  bagimli degiskendeki varyasin yuzde kacini aciklamislardir
        R2_score=cross_val_score(knn_model, X_train , y_train, cv = 10, scoring = "r2").mean()#burada egitim setimiz icerisinden her saferinde farkli fozlemler alindigindan dolayi 
        df1=pd.DataFrame(R2_score, index=["optimum_R2_SCORE"],columns=['Coefficient'])                   #10 adet farkli  r2 score  degeri olustur duk ve bunlarin ortalam degerlerini aldik
        return df1
    
    
    
    def Distributions_and_Variance_of_Test_Residuals(knn_model,   X_test,   y_test,   X_train,   y_train):
        predictions_test = knn_model.predict(X_test)# Olusturdugumuz modelin test degerleri icin olusturdugu tahmini degerler
        residual_test= y_test-predictions_test


        predictions_train = knn_model.predict(X_train)# Olusturdugumuz modelin train degerleri icin olusturdugu tahmini degerler
        residual_train= y_train-predictions_train

        fig, axes = plt.subplots(1,4,figsize=(20,5))

        plt.style.use("seaborn-darkgrid")
        df_test=pd.DataFrame({"Actual_Dependent_Values":y_test,
                             "Predicted_Dependent_Values":predictions_test,
                             "Residuals":residual_test}).reset_index(drop=True)
        
        df_train=pd.DataFrame({"Actual_Dependent_Values":y_train,
                              "Predicted_Dependent_Values":predictions_train,
                              "Residuals":residual_train}).reset_index(drop=True)
              
        fig.suptitle("Distributions and Variance of Residuals ")

        a= "Skewness: %.2f" % residual_test.skew()
        b="Kurtosis: %.2f" % residual_test.kurtosis()
        c="Mean: %.2f" %   residual_test.mean()
        d="Median: %.2f" % residual_test.median()


        sns.distplot(df_test["Residuals"], bins=20,ax=axes[0])
        axes[0].set(title="{} {} {} {}".format(a,b,c,d), 
                    xlabel="Value of Deviations", 
                    ylabel="Frequency") 
     
    

        sns.scatterplot(x="Actual_Dependent_Values", y="Predicted_Dependent_Values",data=df_test, ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                    xlabel=" Actual Test Values", 
                    ylabel="Predicted Test Values")
            


        sns.lineplot(x=df_test.index,y=df_test["Residuals"],data=df_test,ax=axes[2])
        axes[2].set(title="Residuals of Test Values  ", 
                    xlabel="Indexes of Test Values ", 
                    ylabel="Value of Deviations")
        

        sns.lineplot(x=df_train.index,y=df_train["Residuals"],data=df_train,ax=axes[3])
        axes[3].set(title="Residuals of Train Values  ", 
                    xlabel="Indexes of Train Values ", 
                    ylabel="Value of Deviations")
        
        
    def Harmonies_of_Actual_values_with_Predicted_values(knn_model,   X_test,   y_test,   X_train,   y_train):
        fig, axes = plt.subplots(1,2, figsize=(15,7))
        fig.suptitle("Distributions of Test & Train values and their harmonies with Predicted values")
        sns.distplot(y_train, hist=False ,color="r", label="Actual Values",ax=axes[0])
        sns.distplot(knn_model.predict(X_train),hist=False,color="b",label="Predicted Values",ax=axes[0])
        axes[0].set(title="Actual vs Predicted Train Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[0].legend(loc='best')

        sns.distplot(y_test, hist=False ,color="r", label="Actual Value",ax=axes[1])
        sns.distplot(knn_model.predict(X_test),hist=False,color="b",label="Predicted Value",ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[1].legend(loc='best')    
    
    b=optimum_root_mean_square_error( knn_model, X_train, X_test, y_train, y_test)
    c=optimum_R_squred_Score(knn_model ,X_train, y_train)
    d=Distributions_and_Variance_of_Test_Residuals(knn_model,   X_test,   y_test,   X_train,   y_train)
    e=Harmonies_of_Actual_values_with_Predicted_values(knn_model,   X_test,   y_test,   X_train,   y_train)
    
    return display(b),display(c),d ,e

# 2-)Support Vector Regression Model

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')


import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from scipy.stats import kurtosis, skew
from IPython.display import display
plt.style.use("seaborn-darkgrid")

def Support_Vector_Regression_Model(independent_variables, target_variable):
    print("************************************************* Support Vector Regression Model*********************************************************")
    
    def optimum_value_of_complexity_parameter(independent_variables, target_variable):
        X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
        svr_params = {"C": np.linspace(0.1,10000,600)}
        svr_model=SVR()
        svr_cv_model = GridSearchCV(svr_model,svr_params, cv = 10,n_jobs=-1).fit(X_train, y_train)
        optimum_parameter=svr_cv_model.best_params_["C"]
        print("Optimum value of complexity parameter is {} ".format(optimum_parameter))
        return optimum_parameter
    a=optimum_value_of_complexity_parameter(independent_variables, target_variable)
    X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
    svr_model=SVR(C=a).fit(X_train, y_train)
    
    
    def optimum_root_mean_square_error( svr_model, X_train, X_test, y_train, y_test):# Bu kismda  diger  modelerle karsilastirip en iyi modeli secmek icin  opitimum rmse kullanacagiz  
        optimum_rmse_test=np.sqrt(-cross_val_score( svr_model, X_test, y_test, cv = 10, scoring = "neg_mean_squared_error")).mean()#K KATLI cross validtion  uygulama yaptiktan sonnraki TEST hatamiz
        optimum_rmse_train=np.sqrt(-cross_val_score( svr_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")).mean()#K KATLI cross validtion  uygulama yaptiktan sonnraki TRAIN hatamiz
        df1=pd.DataFrame(optimum_rmse_test, index=["optimum_RMSE_test"],columns=['Coefficient'])
        df2=pd.DataFrame(optimum_rmse_train, index=["optimum_RMSE_train"],columns=['Coefficient'])
        df_concat=pd.concat([df2,df1],axis=0)
        return df_concat
    
    def optimum_R_squred_Score(svr_model ,X_train, y_train):#R^2 (R-square degeridir).#Yani bagimsiz degiskenler  bagimli degiskendeki varyasin yuzde kacini aciklamislardir
        R2_score=cross_val_score(svr_model, X_train , y_train, cv = 10, scoring = "r2").mean()#burada egitim setimiz icerisinden her saferinde farkli fozlemler alindigindan dolayi 
        df1=pd.DataFrame(R2_score, index=["optimum_R2_SCORE"],columns=['Coefficient'])                   #10 adet farkli  r2 score  degeri olustur duk ve bunlarin ortalam degerlerini aldik
        return df1
    
    def Distributions_and_Variance_of_Test_Residuals(svr_model,   X_test,   y_test,   X_train,   y_train):
        predictions_test = svr_model.predict(X_test)# Olusturdugumuz modelin test degerleri icin olusturdugu tahmini degerler
        residual_test= y_test-predictions_test


        predictions_train = svr_model.predict(X_train)# Olusturdugumuz modelin train degerleri icin olusturdugu tahmini degerler
        residual_train= y_train-predictions_train

        fig, axes = plt.subplots(1,4,figsize=(20,5))

        plt.style.use("seaborn-darkgrid")
        df_test=pd.DataFrame({"Actual_Dependent_Values":y_test,
                             "Predicted_Dependent_Values":predictions_test,
                             "Residuals":residual_test}).reset_index(drop=True)
        
        df_train=pd.DataFrame({"Actual_Dependent_Values":y_train,
                              "Predicted_Dependent_Values":predictions_train,
                              "Residuals":residual_train}).reset_index(drop=True)
              
        fig.suptitle("Distributions and Variance of Residuals ")

        a= "Skewness: %.2f" % residual_test.skew()
        b="Kurtosis: %.2f" % residual_test.kurtosis()
        c="Mean: %.2f" %   residual_test.mean()
        d="Median: %.2f" % residual_test.median()


        sns.distplot(df_test["Residuals"], bins=20,ax=axes[0])
        axes[0].set(title="{} {} {} {}".format(a,b,c,d), 
                    xlabel="Value of Deviations", 
                    ylabel="Frequency") 
     
    

        sns.scatterplot(x="Actual_Dependent_Values", y="Predicted_Dependent_Values",data=df_test, ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                    xlabel=" Actual Test Values", 
                    ylabel="Predicted Test Values")
            


        sns.lineplot(x=df_test.index,y=df_test["Residuals"],data=df_test,ax=axes[2])
        axes[2].set(title="Residuals of Test Values  ", 
                    xlabel="Indexes of Test Values ", 
                    ylabel="Value of Deviations")
        

        sns.lineplot(x=df_train.index,y=df_train["Residuals"],data=df_train,ax=axes[3])
        axes[3].set(title="Residuals of Train Values  ", 
                    xlabel="Indexes of Train Values ", 
                    ylabel="Value of Deviations")
        
        
    def Harmonies_of_Actual_values_with_Predicted_values(svr_model,   X_test,   y_test,   X_train,   y_train):
        fig, axes = plt.subplots(1,2, figsize=(15,7))
        fig.suptitle("Distributions of Test & Train values and their harmonies with Predicted values")
        sns.distplot(y_train, hist=False ,color="r", label="Actual Values",ax=axes[0])
        sns.distplot(svr_model.predict(X_train),hist=False,color="b",label="Predicted Values",ax=axes[0])
        axes[0].set(title="Actual vs Predicted Train Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[0].legend(loc='best')

        sns.distplot(y_test, hist=False ,color="r", label="Actual Value",ax=axes[1])
        sns.distplot(svr_model.predict(X_test),hist=False,color="b",label="Predicted Value",ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[1].legend(loc='best')   
    
    b=optimum_root_mean_square_error( svr_model, X_train, X_test, y_train, y_test)
    c=optimum_R_squred_Score(svr_model ,X_train, y_train)
    d=Distributions_and_Variance_of_Test_Residuals(svr_model,   X_test,   y_test,   X_train,   y_train)
    e=Harmonies_of_Actual_values_with_Predicted_values(svr_model,   X_test,   y_test,   X_train,   y_train)
    
    return a,display(b),display(c), d, e

# 3-)Multiple Layers Regression Models

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')


import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import kurtosis, skew
from IPython.display import display
plt.style.use("seaborn-darkgrid")

def Multiple_Layers_Regression_Model(independent_variables, target_variable):
    print("*************************************************Multiple Layers Regression Model*********************************************************")
    
    
    def optimum_parameters(independent_variables, target_variable):
        X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
        X_train_scaled=StandardScaler().fit_transform(X_train) 
        X_test_scaled=StandardScaler().fit_transform(X_test)
        mlp_params = {'alpha':[ 0.1, 0.05, 0.025, 0.013 , 0.007, 0.001, 0.0005, 0.0001],
             'hidden_layer_sizes': [(75,75), (50,100) ,(100,200) ,(100,50,150),(100,100,100)],
             'activation': ['relu','logistic']}
        mlp_model = MLPRegressor()
        mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv = 10,n_jobs=-1)
        mlp_cv_model.fit(X_train_scaled, y_train)
        optimum_parameters=mlp_cv_model.best_params_
        print("Optimum model parameters are {}".format(optimum_parameters))
        return optimum_parameters
    
    a=optimum_parameters(independent_variables, target_variable)
    
    X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
    X_train_scaled=StandardScaler().fit_transform(X_train) 
    X_test_scaled=StandardScaler().fit_transform(X_test)
    
    mlp_model = MLPRegressor(alpha = a['alpha'], hidden_layer_sizes = a['hidden_layer_sizes'],activation=a['activation']).fit(X_train_scaled, y_train)
    
    
    def optimum_root_mean_square_error( mlp_model, X_train_scaled, X_test_scaled, y_train, y_test):# Bu kismda  diger  modelerle karsilastirip en iyi modeli secmek icin  opitimum rmse kullanacagiz  
        optimum_rmse_test=np.sqrt(-cross_val_score( mlp_model, X_test_scaled, y_test, cv = 10, scoring = "neg_mean_squared_error")).mean()#K KATLI cross validtion  uygulama yaptiktan sonnraki TEST hatamiz
        optimum_rmse_train=np.sqrt(-cross_val_score( mlp_model, X_train_scaled, y_train, cv = 10, scoring = "neg_mean_squared_error")).mean()#K KATLI cross validtion  uygulama yaptiktan sonnraki TRAIN hatamiz
        df1=pd.DataFrame(optimum_rmse_test, index=["optimum_RMSE_test"],columns=['Coefficient'])
        df2=pd.DataFrame(optimum_rmse_train, index=["optimum_RMSE_train"],columns=['Coefficient'])
        df_concat=pd.concat([df2,df1],axis=0)
        return df_concat
    
    def optimum_R_squred_Score(mlp_model ,X_train_scaled, y_train):#R^2 (R-square degeridir).#Yani bagimsiz degiskenler  bagimli degiskendeki varyasin yuzde kacini aciklamislardir
        R2_score=cross_val_score(mlp_model, X_train_scaled , y_train, cv = 10, scoring = "r2").mean()#burada egitim setimiz icerisinden her saferinde farkli fozlemler alindigindan dolayi 
        df1=pd.DataFrame(R2_score, index=["optimum_R2_SCORE"],columns=['Coefficient'])                   #10 adet farkli  r2 score  degeri olustur duk ve bunlarin ortalam degerlerini aldik
        return df1
    
    
    
    def Distributions_and_Variance_of_Test_Residuals(mlp_model,   X_test_scaled,   y_test,   X_train_scaled,   y_train):
        predictions_test = mlp_model.predict(X_test_scaled)# Olusturdugumuz modelin test degerleri icin olusturdugu tahmini degerler
        residual_test= y_test-predictions_test


        predictions_train = mlp_model.predict(X_train_scaled)# Olusturdugumuz modelin train degerleri icin olusturdugu tahmini degerler
        residual_train= y_train-predictions_train

        fig, axes = plt.subplots(1,4,figsize=(20,5))

        plt.style.use("seaborn-darkgrid")
        df_test=pd.DataFrame({"Actual_Dependent_Values":y_test,
                             "Predicted_Dependent_Values":predictions_test,
                             "Residuals":residual_test}).reset_index(drop=True)
        
        df_train=pd.DataFrame({"Actual_Dependent_Values":y_train,
                              "Predicted_Dependent_Values":predictions_train,
                              "Residuals":residual_train}).reset_index(drop=True)
              
        fig.suptitle("Distributions and Variance of Residuals ")

        a= "Skewness: %.2f" % residual_test.skew()
        b="Kurtosis: %.2f" % residual_test.kurtosis()
        c="Mean: %.2f" %   residual_test.mean()
        d="Median: %.2f" % residual_test.median()


        sns.distplot(df_test["Residuals"], bins=20,ax=axes[0])
        axes[0].set(title="{} {} {} {}".format(a,b,c,d), 
                    xlabel="Value of Deviations", 
                    ylabel="Frequency") 
     
    

        sns.scatterplot(x="Actual_Dependent_Values", y="Predicted_Dependent_Values",data=df_test, ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                    xlabel=" Actual Test Values", 
                    ylabel="Predicted Test Values")
            


        sns.lineplot(x=df_test.index,y=df_test["Residuals"],data=df_test,ax=axes[2])
        axes[2].set(title="Residuals of Test Values  ", 
                    xlabel="Indexes of Test Values ", 
                    ylabel="Value of Deviations")
        

        sns.lineplot(x=df_train.index,y=df_train["Residuals"],data=df_train,ax=axes[3])
        axes[3].set(title="Residuals of Train Values  ", 
                    xlabel="Indexes of Train Values ", 
                    ylabel="Value of Deviations")
        
    def Harmonies_of_Actual_values_with_Predicted_values(mlp_model,   X_test_scaled,   y_test,   X_train_scaled,   y_train):
        fig, axes = plt.subplots(1,2, figsize=(15,7))
        fig.suptitle("Distributions of Test & Train values and their harmonies with Predicted values")
        sns.distplot(y_train, hist=False ,color="r", label="Actual Values",ax=axes[0])
        sns.distplot(mlp_model.predict(X_train_scaled),hist=False,color="b",label="Predicted Values",ax=axes[0])
        axes[0].set(title="Actual vs Predicted Train Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[0].legend(loc='best')

        sns.distplot(y_test, hist=False ,color="r", label="Actual Value",ax=axes[1])
        sns.distplot(mlp_model.predict(X_test_scaled),hist=False,color="b",label="Predicted Value",ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[1].legend(loc='best')
    
    b=optimum_root_mean_square_error( mlp_model, X_train_scaled, X_test_scaled, y_train, y_test)
    c= optimum_R_squred_Score(mlp_model ,X_train_scaled, y_train)
    d=Distributions_and_Variance_of_Test_Residuals(mlp_model,   X_test_scaled,   y_test,   X_train_scaled,   y_train)
    e=Harmonies_of_Actual_values_with_Predicted_values(mlp_model,   X_test_scaled,   y_test,   X_train_scaled,   y_train)
    
    return a ,display(b),display(c),d ,e

# 4-)CART ( Classification and Regressions Trees) Regression Model

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')


import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import kurtosis, skew
from IPython.display import display
plt.style.use("seaborn-darkgrid")


def CART_Regression_Model(independent_variables, target_variable):
    print("*************************************************CART Regression Model*********************************************************")
    
    def optimum_parameters(independent_variables, target_variable):
        X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
        cart_params = {"min_samples_split": range(2,100),
                       "max_leaf_nodes": range(2,10)}
        cart_model = DecisionTreeRegressor()
        cart_cv_model = GridSearchCV(cart_model, cart_params, cv = 10)
        cart_cv_model.fit(X_train, y_train)
        optimum_parameters=cart_cv_model.best_params_
        print("Optimum model parameters are {}".format(optimum_parameters))
        return optimum_parameters
    
    a=optimum_parameters(independent_variables, target_variable)
    X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
    cart_model = DecisionTreeRegressor(max_leaf_nodes=a['max_leaf_nodes'],min_samples_split = a['min_samples_split']).fit(X_train, y_train)
    
    def optimum_root_mean_square_error( cart_model, X_train, X_test, y_train, y_test):
        optimum_rmse_test=np.sqrt(-cross_val_score( cart_model, X_test, y_test, cv = 10, scoring = "neg_mean_squared_error")).mean()
        optimum_rmse_train=np.sqrt(-cross_val_score( cart_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")).mean()
        df1=pd.DataFrame(optimum_rmse_test, index=["optimum_RMSE_test"],columns=['Coefficient'])
        df2=pd.DataFrame(optimum_rmse_train, index=["optimum_RMSE_train"],columns=['Coefficient'])
        df_concat=pd.concat([df2,df1],axis=0)
        return df_concat
    
    def optimum_R_squred_Score(cart_model ,X_train, y_train):
        R2_score=cross_val_score(cart_model, X_train , y_train, cv = 10, scoring = "r2").mean()
        df1=pd.DataFrame(R2_score, index=["optimum_R2_SCORE"],columns=['Coefficient'])                  
        return df1
    
    def Distributions_and_Variance_of_Test_Residuals(cart_model,   X_test,   y_test,   X_train,   y_train):
        predictions_test = cart_model.predict(X_test)
        residual_test= y_test-predictions_test


        predictions_train = cart_model.predict(X_train)
        residual_train= y_train-predictions_train

        fig, axes = plt.subplots(1,4,figsize=(20,5))

        plt.style.use("seaborn-darkgrid")
        df_test=pd.DataFrame({"Actual_Dependent_Values":y_test,
                             "Predicted_Dependent_Values":predictions_test,
                             "Residuals":residual_test}).reset_index(drop=True)
        
        df_train=pd.DataFrame({"Actual_Dependent_Values":y_train,
                              "Predicted_Dependent_Values":predictions_train,
                              "Residuals":residual_train}).reset_index(drop=True)
              
        fig.suptitle("Distributions and Variance of Residuals ")

        a= "Skewness: %.2f" % residual_test.skew()
        b="Kurtosis: %.2f" % residual_test.kurtosis()
        c="Mean: %.2f" %   residual_test.mean()
        d="Median: %.2f" % residual_test.median()


        sns.distplot(df_test["Residuals"], bins=20,ax=axes[0])
        axes[0].set(title="{} {} {} {}".format(a,b,c,d), 
                    xlabel="Value of Deviations", 
                    ylabel="Frequency") 
     
    

        sns.scatterplot(x="Actual_Dependent_Values", y="Predicted_Dependent_Values",data=df_test, ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                    xlabel=" Actual Test Values", 
                    ylabel="Predicted Test Values")
            


        sns.lineplot(x=df_test.index,y=df_test["Residuals"],data=df_test,ax=axes[2])
        axes[2].set(title="Residuals of Test Values  ", 
                    xlabel="Indexes of Test Values ", 
                    ylabel="Value of Deviations")
        

        sns.lineplot(x=df_train.index,y=df_train["Residuals"],data=df_train,ax=axes[3])
        axes[3].set(title="Residuals of Train Values  ", 
                    xlabel="Indexes of Train Values ", 
                    ylabel="Value of Deviations")
        
    def Harmonies_of_Actual_values_with_Predicted_values(cart_model,   X_test,   y_test,   X_train,   y_train):
        fig, axes = plt.subplots(1,2, figsize=(15,7))
        fig.suptitle("Distributions of Test & Train values and their harmonies with Predicted values")
        sns.distplot(y_train, hist=False ,color="r", label="Actual Values",ax=axes[0])
        sns.distplot(cart_model.predict(X_train),hist=False,color="b",label="Predicted Values",ax=axes[0])
        axes[0].set(title="Actual vs Predicted Train Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[0].legend(loc='best')

        sns.distplot(y_test, hist=False ,color="r", label="Actual Value",ax=axes[1])
        sns.distplot(cart_model.predict(X_test),hist=False,color="b",label="Predicted Value",ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[1].legend(loc='best') 
                          
                                                        
                                                        
    b=optimum_root_mean_square_error( cart_model, X_train, X_test, y_train, y_test)
    c=optimum_R_squred_Score(cart_model ,X_train, y_train)
    d=Distributions_and_Variance_of_Test_Residuals(cart_model,   X_test,   y_test,   X_train,   y_train)
    e=Harmonies_of_Actual_values_with_Predicted_values(cart_model,   X_test,   y_test,   X_train,   y_train)
    
    return a, display(b) ,display(c),d,e

# 5-) Bagged Trees Regression Model

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')


import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import kurtosis, skew
from IPython.display import display
plt.style.use("seaborn-darkgrid")





def Bagged_Trees_Regression_Model(independent_variables, target_variable):
    print("*************************************************Bagged Trees  Regression  Model*********************************************************")
    
    def optimum_number_of_trees(independent_variables, target_variable):
        X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
        bag_params = {"n_estimators": range(2,30)}
        bag_model=BaggingRegressor(bootstrap_features = True)
        bag_cv_model = GridSearchCV(bag_model, bag_params, cv = 10)
        bag_cv_model.fit(X_train, y_train)
        optimum_number_of_trees=bag_cv_model.best_params_
        print("Optimum numberof trees  is {}".format( optimum_number_of_trees))
        return optimum_number_of_trees
    
    a=optimum_number_of_trees(independent_variables, target_variable)
    X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
    bag_model= BaggingRegressor(bootstrap_features = True, n_estimators =a['n_estimators'],random_state=101).fit(X_train,y_train)
    
    
    def optimum_root_mean_square_error(  bag_model, X_train, X_test, y_train, y_test):
        optimum_rmse_test=np.sqrt(-cross_val_score( bag_model, X_test, y_test, cv = 10, scoring = "neg_mean_squared_error")).mean()
        optimum_rmse_train=np.sqrt(-cross_val_score( bag_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")).mean()
        df1=pd.DataFrame(optimum_rmse_test, index=["optimum_RMSE_test"],columns=['Coefficient'])
        df2=pd.DataFrame(optimum_rmse_train, index=["optimum_RMSE_train"],columns=['Coefficient'])
        df_concat=pd.concat([df2,df1],axis=0)
        return df_concat
    
    def optimum_R_squred_Score(bag_model ,X_train, y_train):
        R2_score=cross_val_score(bag_model, X_train , y_train, cv = 10, scoring = "r2").mean()
        df1=pd.DataFrame(R2_score, index=["optimum_R2_SCORE"],columns=['Coefficient'])                  
        return df1
    
    def Distributions_and_Variance_of_Test_Residuals(bag_model,   X_test,   y_test,   X_train,   y_train):
        predictions_test = bag_model.predict(X_test)
        residual_test= y_test-predictions_test


        predictions_train = bag_model.predict(X_train)
        residual_train= y_train-predictions_train

        fig, axes = plt.subplots(1,4,figsize=(20,5))

        plt.style.use("seaborn-darkgrid")
        df_test=pd.DataFrame({"Actual_Dependent_Values":y_test,
                             "Predicted_Dependent_Values":predictions_test,
                             "Residuals":residual_test}).reset_index(drop=True)
        
        df_train=pd.DataFrame({"Actual_Dependent_Values":y_train,
                              "Predicted_Dependent_Values":predictions_train,
                              "Residuals":residual_train}).reset_index(drop=True)
              
        fig.suptitle("Distributions and Variance of Residuals ")

        a= "Skewness: %.2f" % residual_test.skew()
        b="Kurtosis: %.2f" % residual_test.kurtosis()
        c="Mean: %.2f" %   residual_test.mean()
        d="Median: %.2f" % residual_test.median()


        sns.distplot(df_test["Residuals"], bins=20,ax=axes[0])
        axes[0].set(title="{} {} {} {}".format(a,b,c,d), 
                    xlabel="Value of Deviations", 
                    ylabel="Frequency") 
     
    

        sns.scatterplot(x="Actual_Dependent_Values", y="Predicted_Dependent_Values",data=df_test, ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                    xlabel=" Actual Test Values", 
                    ylabel="Predicted Test Values")
            


        sns.lineplot(x=df_test.index,y=df_test["Residuals"],data=df_test,ax=axes[2])
        axes[2].set(title="Residuals of Test Values  ", 
                    xlabel="Indexes of Test Values ", 
                    ylabel="Value of Deviations")
        

        sns.lineplot(x=df_train.index,y=df_train["Residuals"],data=df_train,ax=axes[3])
        axes[3].set(title="Residuals of Train Values  ", 
                    xlabel="Indexes of Train Values ", 
                    ylabel="Value of Deviations")
        
        
    def Harmonies_of_Actual_values_with_Predicted_values(bag_model,   X_test,   y_test,   X_train,   y_train):
        fig, axes = plt.subplots(1,2, figsize=(15,7))
        fig.suptitle("Distributions of Test & Train values and their harmonies with Predicted values")
        sns.distplot(y_train, hist=False ,color="r", label="Actual Values",ax=axes[0])
        sns.distplot(bag_model.predict(X_train),hist=False,color="b",label="Predicted Values",ax=axes[0])
        axes[0].set(title="Actual vs Predicted Train Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[0].legend(loc='best')

        sns.distplot(y_test, hist=False ,color="r", label="Actual Value",ax=axes[1])
        sns.distplot(bag_model.predict(X_test),hist=False,color="b",label="Predicted Value",ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[1].legend(loc='best')    
    
    
    
    b=optimum_root_mean_square_error(  bag_model, X_train, X_test, y_train, y_test)
    c=optimum_R_squred_Score(bag_model ,X_train, y_train)
    d=Distributions_and_Variance_of_Test_Residuals(bag_model,   X_test,   y_test,   X_train,   y_train)
    e=Harmonies_of_Actual_values_with_Predicted_values(bag_model,   X_test,   y_test,   X_train,   y_train)
     
    return a ,display(b),display(c),d,e

# 6-)Random Forest Regression Model

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')


import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import kurtosis, skew
from IPython.display import display
plt.style.use("seaborn-darkgrid")

def Random_Forest_Regression_Model(independent_variables, target_variable):
    print("*************************************************Random Forest Regression  Model*********************************************************")
    
    def optimum_parameters(independent_variables, target_variable):
        X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
        rf_params = {'max_depth': list(range(1,20)),
                     'max_features': [3,5,10,15,25,100],
                     'n_estimators' : [10,50,100, 200, 500, 1000, 2000]}
        rf_model = RandomForestRegressor(random_state = 101)
        rf_cv_model = GridSearchCV(rf_model, rf_params, cv = 10, n_jobs = -1)
        rf_cv_model.fit(X_train, y_train)
        optimum_parameters=rf_cv_model.best_params_
        print("Optimum model parameters are {}".format(optimum_parameters))
        return optimum_parameters
    
    a=optimum_parameters(independent_variables, target_variable)
    X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
    rf_model= RandomForestRegressor(random_state = 101,max_depth=a['max_depth'],max_features=a['max_features'],n_estimators=a['n_estimators']).fit(X_train, y_train)
    
    def optimum_root_mean_square_error( rf_model, X_train, X_test, y_train, y_test):
        optimum_rmse_test=np.sqrt(-cross_val_score( rf_model, X_test, y_test, cv = 10, scoring = "neg_mean_squared_error")).mean()
        optimum_rmse_train=np.sqrt(-cross_val_score( rf_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")).mean()
        df1=pd.DataFrame(optimum_rmse_test, index=["optimum_RMSE_test"],columns=['Coefficient'])
        df2=pd.DataFrame(optimum_rmse_train, index=["optimum_RMSE_train"],columns=['Coefficient'])
        df_concat=pd.concat([df2,df1],axis=0)
        return df_concat
    
    def optimum_R_squred_Score(rf_model,X_train, y_train):
        R2_score=cross_val_score(rf_model, X_train , y_train, cv = 10, scoring = "r2").mean()
        df1=pd.DataFrame(R2_score, index=["optimum_R2_SCORE"],columns=['Coefficient'])                  
        return df1
    
    def Distributions_and_Variance_of_Test_Residuals(rf_model,   X_test,   y_test,   X_train,   y_train):
        predictions_test = rf_model.predict(X_test)
        residual_test= y_test-predictions_test


        predictions_train = rf_model.predict(X_train)
        residual_train= y_train-predictions_train

        fig, axes = plt.subplots(1,4,figsize=(20,5))

        plt.style.use("seaborn-darkgrid")
        df_test=pd.DataFrame({"Actual_Dependent_Values":y_test,
                             "Predicted_Dependent_Values":predictions_test,
                             "Residuals":residual_test}).reset_index(drop=True)
        
        df_train=pd.DataFrame({"Actual_Dependent_Values":y_train,
                              "Predicted_Dependent_Values":predictions_train,
                              "Residuals":residual_train}).reset_index(drop=True)
              
        fig.suptitle("Distributions and Variance of Residuals ")

        a= "Skewness: %.2f" % residual_test.skew()
        b="Kurtosis: %.2f" % residual_test.kurtosis()
        c="Mean: %.2f" %   residual_test.mean()
        d="Median: %.2f" % residual_test.median()


        sns.distplot(df_test["Residuals"], bins=20,ax=axes[0])
        axes[0].set(title="{} {} {} {}".format(a,b,c,d), 
                    xlabel="Value of Deviations", 
                    ylabel="Frequency") 
     
    

        sns.scatterplot(x="Actual_Dependent_Values", y="Predicted_Dependent_Values",data=df_test, ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                    xlabel=" Actual Test Values", 
                    ylabel="Predicted Test Values")
            


        sns.lineplot(x=df_test.index,y=df_test["Residuals"],data=df_test,ax=axes[2])
        axes[2].set(title="Residuals of Test Values  ", 
                    xlabel="Indexes of Test Values ", 
                    ylabel="Value of Deviations")
        

        sns.lineplot(x=df_train.index,y=df_train["Residuals"],data=df_train,ax=axes[3])
        axes[3].set(title="Residuals of Train Values  ", 
                    xlabel="Indexes of Train Values ", 
                    ylabel="Value of Deviations")
    
    def Harmonies_of_Actual_values_with_Predicted_values(rf_model,   X_test,   y_test,   X_train,   y_train):
        fig, axes = plt.subplots(1,2, figsize=(15,7))
        fig.suptitle("Distributions of Test & Train values and their harmonies with Predicted values")
        sns.distplot(y_train, hist=False ,color="r", label="Actual Values",ax=axes[0])
        sns.distplot(rf_model.predict(X_train),hist=False,color="b",label="Predicted Values",ax=axes[0])
        axes[0].set(title="Actual vs Predicted Train Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[0].legend(loc='best')

        sns.distplot(y_test, hist=False ,color="r", label="Actual Value",ax=axes[1])
        sns.distplot(rf_model.predict(X_test),hist=False,color="b",label="Predicted Value",ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[1].legend(loc='best')
    
    
    
    
    b=optimum_root_mean_square_error( rf_model, X_train, X_test, y_train, y_test)
    c=optimum_R_squred_Score(rf_model,X_train, y_train)
    d=Distributions_and_Variance_of_Test_Residuals(rf_model,   X_test,   y_test,   X_train,   y_train)
    e=Harmonies_of_Actual_values_with_Predicted_values(rf_model,   X_test,   y_test,   X_train,   y_train)
    
    return a,display(b),display(c),d,e

# 7-)Gradient Boosting Machines (GBM)  Regression model

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')


import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import kurtosis, skew
from IPython.display import display
plt.style.use("seaborn-darkgrid")



def Gradient_Boosting_Machine_Regression_Model(independent_variables, target_variable):
    print("*************************************************Gradient Boosting Machine RegressionModel*********************************************************")
    
    def optimum_parameters(independent_variables, target_variable):
        X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
        gbm_params = {'learning_rate': [0.0001,0.001, 0.01, 0.1, 0.2, 0.5, 1, 10],
                      'max_depth': [3, 5, 8,50,100,500],
                      'n_estimators': [200, 500, 1000, 2000],
                      'subsample': [1,0.5,0.75]}
        
        gbm = GradientBoostingRegressor()
        gbm_cv_model = GridSearchCV(gbm, gbm_params, cv = 10, n_jobs = -1)
        gbm_cv_model.fit(X_train, y_train)
        optimum_parameters=gbm_cv_model.best_params_
        print("Optimum model parameters are {}".format(optimum_parameters))
        return optimum_parameters
    
    a=optimum_parameters(independent_variables, target_variable)
    X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
    gbm_model=GradientBoostingRegressor(learning_rate=a['learning_rate'],max_depth=a['max_depth'],n_estimators=a['n_estimators'],subsample=a['subsample']).fit(X_train,y_train)
    
    def optimum_root_mean_square_error( gbm_model, X_train, X_test, y_train, y_test):
        optimum_rmse_test=np.sqrt(-cross_val_score( gbm_model, X_test, y_test, cv = 10, scoring = "neg_mean_squared_error")).mean()
        optimum_rmse_train=np.sqrt(-cross_val_score( gbm_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")).mean()
        df1=pd.DataFrame(optimum_rmse_test, index=["optimum_RMSE_test"],columns=['Coefficient'])
        df2=pd.DataFrame(optimum_rmse_train, index=["optimum_RMSE_train"],columns=['Coefficient'])
        df_concat=pd.concat([df2,df1],axis=0)
        return df_concat
    
    def optimum_R_squred_Score(gbm_model,X_train, y_train):
        R2_score=cross_val_score(gbm_model, X_train , y_train, cv = 10, scoring = "r2").mean()
        df1=pd.DataFrame(R2_score, index=["optimum_R2_SCORE"],columns=['Coefficient'])                  
        return df1
    
    def Distributions_and_Variance_of_Test_Residuals(gbm_model,   X_test,   y_test,   X_train,   y_train):
        predictions_test = gbm_model.predict(X_test)
        residual_test= y_test-predictions_test


        predictions_train = gbm_model.predict(X_train)
        residual_train= y_train-predictions_train

        fig, axes = plt.subplots(1,4,figsize=(20,5))

        plt.style.use("seaborn-darkgrid")
        df_test=pd.DataFrame({"Actual_Dependent_Values":y_test,
                             "Predicted_Dependent_Values":predictions_test,
                             "Residuals":residual_test}).reset_index(drop=True)
        
        df_train=pd.DataFrame({"Actual_Dependent_Values":y_train,
                              "Predicted_Dependent_Values":predictions_train,
                              "Residuals":residual_train}).reset_index(drop=True)
              
        fig.suptitle("Distributions and Variance of Residuals ")

        a= "Skewness: %.2f" % residual_test.skew()
        b="Kurtosis: %.2f" % residual_test.kurtosis()
        c="Mean: %.2f" %   residual_test.mean()
        d="Median: %.2f" % residual_test.median()


        sns.distplot(df_test["Residuals"], bins=20,ax=axes[0])
        axes[0].set(title="{} {} {} {}".format(a,b,c,d), 
                    xlabel="Value of Deviations", 
                    ylabel="Frequency") 
     
    

        sns.scatterplot(x="Actual_Dependent_Values", y="Predicted_Dependent_Values",data=df_test, ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                    xlabel=" Actual Test Values", 
                    ylabel="Predicted Test Values")
            


        sns.lineplot(x=df_test.index,y=df_test["Residuals"],data=df_test,ax=axes[2])
        axes[2].set(title="Residuals of Test Values  ", 
                    xlabel="Indexes of Test Values ", 
                    ylabel="Value of Deviations")
        

        sns.lineplot(x=df_train.index,y=df_train["Residuals"],data=df_train,ax=axes[3])
        axes[3].set(title="Residuals of Train Values  ", 
                    xlabel="Indexes of Train Values ", 
                    ylabel="Value of Deviations")
        
    def Harmonies_of_Actual_values_with_Predicted_values(gbm_model,   X_test,   y_test,   X_train,   y_train):
        fig, axes = plt.subplots(1,2, figsize=(15,7))
        fig.suptitle("Distributions of Test & Train values and their harmonies with Predicted values")
        sns.distplot(y_train, hist=False ,color="r", label="Actual Values",ax=axes[0])
        sns.distplot(gbm_model.predict(X_train),hist=False,color="b",label="Predicted Values",ax=axes[0])
        axes[0].set(title="Actual vs Predicted Train Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[0].legend(loc='best')

        sns.distplot(y_test, hist=False ,color="r", label="Actual Value",ax=axes[1])
        sns.distplot(gbm_model.predict(X_test),hist=False,color="b",label="Predicted Value",ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[1].legend(loc='best')
    
    
    
    
    b=optimum_root_mean_square_error( gbm_model, X_train, X_test, y_train, y_test)
    c=optimum_R_squred_Score(gbm_model,X_train, y_train)
    d= Distributions_and_Variance_of_Test_Residuals(gbm_model,   X_test,   y_test,   X_train,   y_train)
    e=Harmonies_of_Actual_values_with_Predicted_values(gbm_model,   X_test,   y_test,   X_train,   y_train)
    
    
    return a, display(b),display(c),d,e

# 8-)Extreme Gradient Boosting (XGBoost) Regression Model

In [ ]:
!pip install xgboost
from warnings import filterwarnings
filterwarnings('ignore')


import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from scipy.stats import kurtosis, skew
from IPython.display import display
plt.style.use("seaborn-darkgrid")



def Extreme_Gradient_Boosting_Regression_Model(independent_variables, target_variable):
    print("************************************************* Extreme Gradient Boosting  Regression Model******************************************************")
    
    def optimum_parameters(independent_variables, target_variable):
        X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
        xgb_grid = {'colsample_bytree': [0.2,0.4,0.6,0.9, 1, 1.2,1.6,2], 
                    'n_estimators':[100, 200, 500, 1000, 5000],
                    'max_depth': [2,3,4,5,6,7,8,9],
                    'learning_rate': [0.1, 0.01, 0.5,0.001]}
        xgb = XGBRegressor()
        xgb_cv = GridSearchCV(xgb, param_grid = xgb_grid, cv = 10, n_jobs = -1)
        xgb_cv.fit(X_train, y_train)
        optimum_parameters=xgb_cv.best_params_
        print("Optimum model parameters are {}".format(optimum_parameters))
        return optimum_parameters
    
    a=optimum_parameters(independent_variables, target_variable)
    X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
    xgb_model=XGBRegressor(colsample_bytree=a['colsample_bytree'],max_depth=a['max_depth'],n_estimators=a['n_estimators'],learning_rate=a['learning_rate']).fit(X_train,y_train)
    
    def optimum_root_mean_square_error( xgb_model, X_train, X_test, y_train, y_test):
        optimum_rmse_test=np.sqrt(-cross_val_score( xgb_model, X_test, y_test, cv = 10, scoring = "neg_mean_squared_error")).mean()
        optimum_rmse_train=np.sqrt(-cross_val_score( xgb_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")).mean()
        df1=pd.DataFrame(optimum_rmse_test, index=["optimum_RMSE_test"],columns=['Coefficient'])
        df2=pd.DataFrame(optimum_rmse_train, index=["optimum_RMSE_train"],columns=['Coefficient'])
        df_concat=pd.concat([df2,df1],axis=0)
        return df_concat
    
    def optimum_R_squred_Score(xgb_model,X_train, y_train):
        R2_score=cross_val_score(xgb_model, X_train , y_train, cv = 10, scoring = "r2").mean()
        df1=pd.DataFrame(R2_score, index=["optimum_R2_SCORE"],columns=['Coefficient'])                  
        return df1
    
    
    
    def Distributions_and_Variance_of_Test_Residuals(xgb_model,   X_test,   y_test,   X_train,   y_train):
        predictions_test = xgb_model.predict(X_test)
        residual_test= y_test-predictions_test


        predictions_train = xgb_model.predict(X_train)
        residual_train= y_train-predictions_train

        fig, axes = plt.subplots(1,4,figsize=(20,5))

        plt.style.use("seaborn-darkgrid")
        df_test=pd.DataFrame({"Actual_Dependent_Values":y_test,
                             "Predicted_Dependent_Values":predictions_test,
                             "Residuals":residual_test}).reset_index(drop=True)
        
        df_train=pd.DataFrame({"Actual_Dependent_Values":y_train,
                              "Predicted_Dependent_Values":predictions_train,
                              "Residuals":residual_train}).reset_index(drop=True)
              
        fig.suptitle("Distributions and Variance of Residuals ")

        a= "Skewness: %.2f" % residual_test.skew()
        b="Kurtosis: %.2f" % residual_test.kurtosis()
        c="Mean: %.2f" %   residual_test.mean()
        d="Median: %.2f" % residual_test.median()


        sns.distplot(df_test["Residuals"], bins=20,ax=axes[0])
        axes[0].set(title="{} {} {} {}".format(a,b,c,d), 
                    xlabel="Value of Deviations", 
                    ylabel="Frequency") 
     
    

        sns.scatterplot(x="Actual_Dependent_Values", y="Predicted_Dependent_Values",data=df_test, ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                    xlabel=" Actual Test Values", 
                    ylabel="Predicted Test Values")
            


        sns.lineplot(x=df_test.index,y=df_test["Residuals"],data=df_test,ax=axes[2])
        axes[2].set(title="Residuals of Test Values  ", 
                    xlabel="Indexes of Test Values ", 
                    ylabel="Value of Deviations")
        

        sns.lineplot(x=df_train.index,y=df_train["Residuals"],data=df_train,ax=axes[3])
        axes[3].set(title="Residuals of Train Values  ", 
                    xlabel="Indexes of Train Values ", 
                    ylabel="Value of Deviations")
        
        
    def Harmonies_of_Actual_values_with_Predicted_values(xgb_model,   X_test,   y_test,   X_train,   y_train):
        fig, axes = plt.subplots(1,2, figsize=(15,7))
        fig.suptitle("Distributions of Test & Train values and their harmonies with Predicted values")
        sns.distplot(y_train, hist=False ,color="r", label="Actual Values",ax=axes[0])
        sns.distplot(xgb_model.predict(X_train),hist=False,color="b",label="Predicted Values",ax=axes[0])
        axes[0].set(title="Actual vs Predicted Train Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[0].legend(loc='best')

        sns.distplot(y_test, hist=False ,color="r", label="Actual Value",ax=axes[1])
        sns.distplot(xgb_model.predict(X_test),hist=False,color="b",label="Predicted Value",ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[1].legend(loc='best')
    
    
    b=optimum_root_mean_square_error( xgb_model, X_train, X_test, y_train, y_test)
    c=optimum_R_squred_Score(xgb_model,X_train, y_train)
    d=Distributions_and_Variance_of_Test_Residuals(xgb_model,   X_test,   y_test,   X_train,   y_train)
    e=Harmonies_of_Actual_values_with_Predicted_values(xgb_model,   X_test,   y_test,   X_train,   y_train)
    
    return  a,display(b),display(c),d,e

# 9-)Light GBM  Regression Model

In [ ]:
!pip install lightgbm
from warnings import filterwarnings
filterwarnings('ignore')


import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from lightgbm import LGBMRegressor
from scipy.stats import kurtosis, skew
from IPython.display import display
plt.style.use("seaborn-darkgrid")


def Light_GBM_Regression_Model(independent_variables, target_variable):
    print("************************************************* Light GBM  Regression Model*********************************************************")
    
    def optimum_parameters(independent_variables, target_variable):
        X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
        lgbm_grid = {'colsample_bytree': [0.2,0.4,0.6,0.9, 1, 1.2,1.6,2],
                     'learning_rate': [0.1, 0.01, 0.5,0.001],
                     'n_estimators': [100, 200, 500, 1000, 5000],
                     'max_depth': [2,3,4,5,6,7,8,9] }
        lgbm = LGBMRegressor()
        lgbm_cv_model = GridSearchCV(lgbm, param_grid = lgbm_grid, cv=10, n_jobs = -1)
        lgbm_cv_model.fit(X_train,y_train)
        optimum_parameters=lgbm_cv_model.best_params_
        print("Optimum model parameters are {}".format(optimum_parameters))
        return optimum_parameters
    
    a=optimum_parameters(independent_variables, target_variable)
    X_train, X_test, y_train, y_test = train_test_split(independent_variables, target_variable, test_size = 0.25, random_state= 101)
    lgbm_model=LGBMRegressor(colsample_bytree=a['colsample_bytree'],max_depth=a['max_depth'],n_estimators=a['n_estimators'],learning_rate=a['learning_rate']).fit(X_train,y_train)
    
    
    
    
    def optimum_root_mean_square_error( lgbm_model, X_train, X_test, y_train, y_test):
        optimum_rmse_test=np.sqrt(-cross_val_score( lgbm_model, X_test, y_test, cv = 10, scoring = "neg_mean_squared_error")).mean()
        optimum_rmse_train=np.sqrt(-cross_val_score( lgbm_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")).mean()
        df1=pd.DataFrame(optimum_rmse_test, index=["optimum_RMSE_test"],columns=['Coefficient'])
        df2=pd.DataFrame(optimum_rmse_train, index=["optimum_RMSE_train"],columns=['Coefficient'])
        df_concat=pd.concat([df2,df1],axis=0)
        return df_concat
    
    def optimum_R_squred_Score(lgbm_model,X_train, y_train):
        R2_score=cross_val_score(lgbm_model, X_train , y_train, cv = 10, scoring = "r2").mean()
        df1=pd.DataFrame(R2_score, index=["optimum_R2_SCORE"],columns=['Coefficient'])                  
        return df1
    
    
    def Distributions_and_Variance_of_Test_Residuals(lgbm_model,   X_test,   y_test,   X_train,   y_train):
        predictions_test = lgbm_model.predict(X_test)
        residual_test= y_test-predictions_test


        predictions_train = lgbm_model.predict(X_train)
        residual_train= y_train-predictions_train

        fig, axes = plt.subplots(1,4,figsize=(20,5))

        plt.style.use("seaborn-darkgrid")
        df_test=pd.DataFrame({"Actual_Dependent_Values":y_test,
                             "Predicted_Dependent_Values":predictions_test,
                             "Residuals":residual_test}).reset_index(drop=True)
        
        df_train=pd.DataFrame({"Actual_Dependent_Values":y_train,
                              "Predicted_Dependent_Values":predictions_train,
                              "Residuals":residual_train}).reset_index(drop=True)
              
        fig.suptitle("Distributions and Variance of Residuals ")

        a= "Skewness: %.2f" % residual_test.skew()
        b="Kurtosis: %.2f" % residual_test.kurtosis()
        c="Mean: %.2f" %   residual_test.mean()
        d="Median: %.2f" % residual_test.median()


        sns.distplot(df_test["Residuals"], bins=20,ax=axes[0])
        axes[0].set(title="{} {} {} {}".format(a,b,c,d), 
                    xlabel="Value of Deviations", 
                    ylabel="Frequency") 
     
    

        sns.scatterplot(x="Actual_Dependent_Values", y="Predicted_Dependent_Values",data=df_test, ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                    xlabel=" Actual Test Values", 
                    ylabel="Predicted Test Values")
            


        sns.lineplot(x=df_test.index,y=df_test["Residuals"],data=df_test,ax=axes[2])
        axes[2].set(title="Residuals of Test Values  ", 
                    xlabel="Indexes of Test Values ", 
                    ylabel="Value of Deviations")
        

        sns.lineplot(x=df_train.index,y=df_train["Residuals"],data=df_train,ax=axes[3])
        axes[3].set(title="Residuals of Train Values  ", 
                    xlabel="Indexes of Train Values ", 
                    ylabel="Value of Deviations")
        
        
    def Harmonies_of_Actual_values_with_Predicted_values(lgbm_model,   X_test,   y_test,   X_train,   y_train):
        fig, axes = plt.subplots(1,2, figsize=(15,7))
        fig.suptitle("Distributions of Test & Train values and their harmonies with Predicted values")
        sns.distplot(y_train, hist=False ,color="r", label="Actual Values",ax=axes[0])
        sns.distplot(lgbm_model.predict(X_train),hist=False,color="b",label="Predicted Values",ax=axes[0])
        axes[0].set(title="Actual vs Predicted Train Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[0].legend(loc='best')

        sns.distplot(y_test, hist=False ,color="r", label="Actual Value",ax=axes[1])
        sns.distplot(lgbm_model.predict(X_test),hist=False,color="b",label="Predicted Value",ax=axes[1])
        axes[1].set(title="Actual vs Predicted Test Values", 
                xlabel=y_train.to_frame().columns[0], 
                ylabel="Proportion")
        axes[1].legend(loc='best')
    
    
    
    b=optimum_root_mean_square_error( lgbm_model, X_train, X_test, y_train, y_test)
    c=optimum_R_squred_Score(lgbm_model,X_train, y_train)
    d=Distributions_and_Variance_of_Test_Residuals(lgbm_model,   X_test,   y_test,   X_train,   y_train)
    e=Harmonies_of_Actual_values_with_Predicted_values(lgbm_model,   X_test,   y_test,   X_train,   y_train)
    
    return a ,display(b), display(c),d,e

# Codes are running

In [ ]:
K_Nearest_Neighbors_Regression_Model(independent_variables, target_variable)

In [ ]:
Support_Vector_Regression_Model(independent_variables, target_variable)

In [ ]:
Multiple_Layers_Regression_Model(independent_variables, target_variable)

In [ ]:
CART_Regression_Model(independent_variables, target_variable)

In [ ]:
Bagged_Trees_Regression_Model(independent_variables, target_variable)

In [ ]:
Random_Forest_Regression_Model(independent_variables, target_variable)

In [ ]:
Gradient_Boosting_Machine_Regression_Model(independent_variables, target_variable)

In [ ]:
Extreme_Gradient_Boosting_Regression_Model(independent_variables, target_variable)

In [ ]:
Light_GBM_Regression_Model(independent_variables, target_variable)